In [51]:
import openai
openai.api_key = "sk-Qgz3rmm3AbTBIwPjA0voHN3llTPWlsBPsNc8fUAd"

In [52]:
import os
import sys
sys.path.append(os.getcwd())

from api import GPT, Example, UIConfig
from api import demo_web_app
from glob import glob
def read_file(path_to_file):
	retval = ""
	file = open(path_to_file)
	retval = file.readlines()
	file.close()
	#print(retval)
	#MAke sure the new line character is not read it throws the model off     
	retval = [x.split("/n")[0][:-1] for x in retval]
	#print(retval)
	return retval

#"""
# Construct GPT object and show some examples
gpt = GPT(engine="curie",
          temperature=0.3,
          max_tokens=300)
#"""

# reade file and convert it to source string and target string tuples
source_names = [item for item in sorted(glob("./Training_Example/sources/*"))]
target_names = [item for item in sorted(glob("./Training_Example/latex/*"))]

#print(source_names)

# open each file in the Training_Example directory
for src_path, target_path in zip(source_names,target_names):
	
	# For each files read the RAW and corrosponding Latex Code
	src_RAW = read_file(src_path)
	target_RAW = read_file(target_path)
	
    # for each pair of RAW and latex prime the GPT model
	if len(src_RAW) != len(target_RAW):
		raise Exception("Source and Latex have mismached number of line {} {} in file {} and {}".format(str(len(src_RAW)), str(len(target_RAW)),src_path,target_path))
		
	for s_RAW, t_RAW in zip(src_RAW,target_RAW):
		gpt.add_example(Example(s_RAW,t_RAW))
		
		print("Source: ", s_RAW)
		print("Output: ", t_RAW)
		print("----")
		

"""
# Define UI configuration
config = UIConfig(description="Text to Manim Class",
                  button_text="Convert",
                  placeholder="Type Raw Text here")

demo_web_app(gpt, config)
"""
print("")

Source:  Question == solve y" + y = 0
Output:  Question \ == \ solve \ y" \ + \ y \ = \ 0
----
Source:  Input,  :=: 
Output:  Input, \ :=: \ 
----
Source:  solve y''(x) + y(x) = 0
Output:  solve \ y''\left(x\right) \ + \ y\left(x\right) \ = \ 0
----
Source:  Result,  :=: 
Output:  Result, \ :=: \ 
----
Source:  y(x) = c_2 sin(x) + c_1 cos(x)
Output:  y\left(x\right) = c_2 \sin\left(x\right) + c_1 cos\left(x\right)
----
Source:  Result, Possible intermediate steps :=: 
Output:  Result, \ Possible \ intermediate \ steps \ :=: \ 
----
Source:  Solve ( d^2 y(x))/( dx^2) + y(x) = 0:
Output:  Solve \ \frac{d^2 \ y\left(x\right)}{dx^2} \ + \ y\left(x\right) \ = \ 0:
----
Source:  Assume a solution will be proportional to e^(λ x) for some constant λ.
Output:  Assume \ a \ solution \ will \ be \ proportional \ to \ e^\left(\lambda \ x\right) \ for \ some \ constant \ \lambda.
----
Source:  Substitute y(x) = e^(λ x) into the differential equation:
Output:  Substitute \ y\left(x\right) \ = \ e^\l

In [53]:
## Testing Script
testFiles = [item for item in sorted(glob("./Training_Example/Test/*"))]
for file in testFiles:
    test_RAW = read_file(file)
    response = []
    for line in test_RAW:
        print("Prompt: ", line)
        t = gpt.get_top_reply(line)
        print(t,end="----")
        response.append(t)


Prompt:  solve x + 7 = -11
output: solve \ x + 7 = -11

----Prompt:  Result,  :=: 
output: Result, \ :=: \ 

----Prompt:  x = -18
output: x = -18

----Prompt:  Result, Possible intermediate steps :=: Solve for x:
output: Result, \ Possible \ intermediate \ steps \ :=: Solve \ for \ x:

----Prompt:  x + 7 = -11
output: x + 7 = -11

----Prompt:  Subtract 7 from both sides:
output: Subtract \ 7 \ from \ both \ sides:

----Prompt:  x + (7 - 7) = -11 - 7
output: x + (7 - 7) = -11 - 7

----Prompt:  7 - 7 = 0:
output: 7 - 7 = 0:

----Prompt:  x = -11 - 7
output: x = -11 - 7

----Prompt:  -11 - 7 = -18:
output: -11 - 7 = -18

----Prompt:  Answer: |  | x = -1
output: Answer \ : | \ | x = -1

----

In [57]:
latex_code = []
for line in response:
    latex_code.append(line.split("\n")[0][7:] + "\n")

In [58]:
import os 
os.system('export PATH=$PATH:/home/mrselukar/manim/manim')

0

In [59]:
for line in latex_code:
    print(line, end="")

 solve \ x + 7 = -11
 Result, \ :=: \ 
 x = -18
 Result, \ Possible \ intermediate \ steps \ :=: Solve \ for \ x:
 x + 7 = -11
 Subtract \ 7 \ from \ both \ sides:
 x + (7 - 7) = -11 - 7
 7 - 7 = 0:
 x = -11 - 7
 -11 - 7 = -18
 Answer \ : | \ | x = -1


In [60]:
from app import latex2Manim
manim_code = latex2Manim.latex2Manim(latex_code)
print(manim_code)

from manimlib.imports import *
class Solution(Scene):
	def construct(self):
		Solve = TexMobject(r" solve \ x + 7 = -11")
		Solve.to_edge(UP)
		self.play(Write(Solve))
		self.wait(2)
		R0 = TexMobject(r" Result, \ :=: \ ")
		self.play(Write(R0))
		self.wait(2)
		R1 = TexMobject(r" x = -18")
		self.play(Transform(R0,R1))
		self.wait(2)
		R1 = TexMobject(r" Result, \ Possible \ intermediate \ steps \ :=: Solve \ for \ x:")
		self.play(Transform(R0,R1))
		self.wait(2)
		R1 = TexMobject(r" x + 7 = -11")
		self.play(Transform(R0,R1))
		self.wait(2)
		R1 = TexMobject(r" Subtract \ 7 \ from \ both \ sides:")
		self.play(Transform(R0,R1))
		self.wait(2)
		R1 = TexMobject(r" x + (7 - 7) = -11 - 7")
		self.play(Transform(R0,R1))
		self.wait(2)
		R1 = TexMobject(r" 7 - 7 = 0:")
		self.play(Transform(R0,R1))
		self.wait(2)
		R1 = TexMobject(r" x = -11 - 7")
		self.play(Transform(R0,R1))
		self.wait(2)
		R1 = TexMobject(r" -11 - 7 = -18")
		self.play(Transform(R0,R1))
		self.wait(2)
		R1 = TexMobje

In [61]:
fptr =  open("solution.py", "w") 
fptr.write(manim_code)
fptr.close()

In [62]:
!python3 /home/mrselukar/manim/manim/manim.py solution.py Solution -pl

Media will be written to ./media/. You can change this behavior with the --media_dir flag.
Writing "solve \ x + 7 = -11" to ./media/Tex/35d2bee847be1d56.tex
Writing "Result, \ :=: \" to ./media/Tex/703be77269f65369.tex                   
Writing "x = -18" to ./media/Tex/d6aefd68dfe210c9.tex                           
Writing "Result, \ Possible \ intermediate \ steps \ :=: Solve \ for \ x:" to ./media/Tex/69ce300714c92ba6.tex
Writing "x + 7 = -11" to ./media/Tex/f8eda7401402683a.tex                       
Writing "Subtract \ 7 \ from \ both \ sides:" to ./media/Tex/668aa58c264deea0.tex
Writing "x + (7 - 7) = -11 - 7" to ./media/Tex/355718587f721450.tex             
Writing "7 - 7 = 0:" to ./media/Tex/dcb2f0a589dec4e2.tex                        
Writing "x = -11 - 7" to ./media/Tex/77dfad41d1288472.tex                       
Writing "-11 - 7 = -18" to ./media/Tex/5812dff072ddf406.tex                     
Writing "Answer \ : | \ | x = -1" to ./media/Tex/f8de35e6f4057b8f.tex           
  

In [ ]:
"""target_names = [item for item in sorted(glob("./Training_Example/latex/*"))]
retval = ""
for path_to_file in target_names:
    print(path_to_file)
    temp = input()
    if temp == "yes":

        file = open(path_to_file)
        retval = file.readlines()
        file.close()
        for i in range(len(retval)):
            print(retval[i])
            x = input()
            if x != '0':
                retval[i] = retval[i].replace(" ", " \ ")
        retval = [x.split("/n")[0][:-1] for x in retval]
    #print(retval)

"""

In [ ]:
"""for l in retval:
    print (l)"""